# Aula 18: transactions

Desejamos construir uma aplicação de empréstimos peer-to-peer simples. Nesta aplicação os usuários podem:
 
- Depositar uma certa quantia em sua conta;
- Sacar da conta, desde que tenham saldo;
- Criar um "emprestimo" (mais como uma linha de credito, realmente) entre dois usuarios
- Pagar (parcial ou totalmente) um emprestimo
- Tomar mais dinheiro ainda de um emprestimo (ou linha de credito)

Mais ainda, temos o requisito de que todas as operações devem ser armazenadas no banco de dados.

Vamos agrupar esses requerimentos como segue:

- Movimentação de conta: depositar ou sacar da conta;
- Criação de empréstimo;
- Operação em empréstimo: adicionar valor (como quando o devedor saca da linha de credito, ou quando os juros são computados), remover valor (como quando o devedor paga alguma quantia).

A modelagem relacional deste problema resultou no seguinte diagrama:

<img src='diagrama.png' alt='Diagrama do modelo relacional' style='width: 500px;'/>


que traduzimos no script de criação da base de dados como segue:

---
```SQL
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);

```
---

Rode esse script ('`emprestimos.sql`') para criar a base de dados no MySQL. Vamos também trazer o código de conexão à base de dados:

In [2]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='emprestimos',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Transactions

Uma *transação* é um grupo de operações na base de dados, em uma dada *sessão*. Vamos conhecer o conceito de transação com as atividades a seguir.

### `COMMIT`

Vamos gerar alguns usuários na nossa base:

In [3]:
db('''
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (1, 'Juca', 'Silva'), 
    (2, 'Mario', 'Ferreira'), 
    (3, 'Ana', 'Soares'), 
    (4, 'Antonio', 'Reis'), 
    (5, 'Paulo', 'Oliveira')
''')

Executando query:


Vamos verificar se os usuários foram inseridos:

In [4]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


Excelente, parece que deu tudo certo! Agora vamos verificar diretamente no MySQL Workbench, só para ter certeza:

```SQL
SELECT * FROM usuario;
```

<img src='usuarios_null.png' alt='Nenhum usuario na tabela!' style='width: 400px;'/>

Epa, cade os usuarios?

Para entender o que aconteceu, precisamos compreender melhor como funciona uma conexão com o banco de dados. Quando criamos um objeto de conexão, estamos iniciando uma *sessão* do banco de dados. Em uma sessão as várias operações sendo realizadas configuram uma *transação*. 

As transações não são efetuadas diretamente no armazenamento permanente, a não ser que o banco tenha a propriedade de *'auto-commit'*. Vamos verificar se esse é o caso na nossa conexão:

In [5]:
print(connection.autocommit)

False


Você deve ver o valor 'False' acima. Isso significa que devemos explicitamente indicar quando estamos satisfeitos com uma transação, para que os valores inseridos (ou removidos) sejam efetivamente registrados no banco de dados. Isso é feito através do comando `COMMIT`:

In [6]:
db('COMMIT')

Executando query:


Agora consulte o banco de dados via MySQL Workbench (ou seja, usando uma sessão diferente da nossa):


```SQL
SELECT * FROM usuario;
```

<img src='usuarios_commit.png' alt='Agora tem usuario na tabela!' style='width: 400px;'/>

Pronto, agora temos usuarios no nosso banco de dados!

Outra forma de indicar o 'commit' da transação é usar o método `commit()` do objeto `connection`:

```Python
connection.commit()
```

### Atividade

Crie os usuários 'Carla Nakamura' e 'Maria Fontana'.

In [8]:
db('''
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (6, 'Carla', 'Nakamura'),
    (7, 'Mario', 'Fontana')
''')

Executando query:


In [9]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))


In [10]:
db('COMMIT')

Executando query:


### ROLLBACK

Para efeito de testes, vamos inserir mais um usuario:

In [16]:
db(''' 
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (8, 'Max', 'Verstappen'),
    (9, 'Lando', 'Norris')
''')



Executando query:


Verificando o resultado da sessão, temos:

In [17]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))
(8, 'Max', 'Verstappen', Decimal('0.00'))
(9, 'Lando', 'Norris', Decimal('0.00'))


Agora, se executarmos o `COMMIT` a nossa modificação se tornará permanente. Porém suponha que estamos arrependidos, e não queremos prosseguir com o `COMMIT` da transação, o que fazer?

Para abandonar as mudanças da transação atual, usamos o comando `ROLLBACK`:

In [18]:
db('ROLLBACK')

Executando query:


Verificando o estado do banco de dados na sessão atual temos:

In [19]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))


Observe que o *rollback* só volta até o último *commit*, e a repetição de *rollbacks* NÃO volta mais ainda para o passado! `COMMIT` e `ROLLBACK` são definitivos!

O *rollback* pode ser obtido também com o método `rollback()` do objeto `connection`:

```Python
connection.rollback()
```

### Atividade

Insira o usuario 'Walter Branco' mas não execute um `COMMIT`, e sim um `ROLLBACK`. Verifique que o `ROLLBACK` funcionou.

In [20]:
db('INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES (8, "Walter", "Branco")')

Executando query:


In [21]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))
(8, 'Walter', 'Branco', Decimal('0.00'))


In [22]:
db('ROLLBACK')

Executando query:


In [23]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))


### `START TRANSACTION`

Para garantir que o banco de dados MySQL não está operando em modo *auto-commit* use o comando `START TRANSACTION`. Este comando garante que estamos iniciando uma transação sem *auto-commit*. Por exemplo:

In [26]:
db('START TRANSACTION')
db('INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES (8, "Carlos", "Sainz")')
db('COMMIT')


Executando query:
Executando query:
Executando query:


In [27]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))
(8, 'Carlos', 'Sainz', Decimal('0.00'))


Vamos reverter essa inserção em uma outra transação.

In [35]:
db('START TRANSACTION')
db('''
SELECT 
    id_usuario INTO @id_usuario_delete
FROM 
    usuario
WHERE
    nome = 'Carlos' AND sobrenome = 'Sainz'
LIMIT
    1
''')
db('SELECT @id_usuario_delete')

Executando query:
Executando query:
Executando query:
(8,)


In [36]:
db('DELETE FROM usuario WHERE id_usuario = @id_usuario_delete')

Executando query:


In [37]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))


In [38]:
db('COMMIT')

Executando query:


### Usando `COMMIT` e `ROLLBACK` para garantir atomicidade de transações

Vamos agora juntar os vários conceitos em um código Python para adicionar itens na tabela `emprestimo`. 

Como podemos ver no modelo E-R devemos informar os ids do credor e do devedor ao inserir uma nova linha nesta tabela, e estes ids devem ser ids válidos de clientes do nosso sistema. Logo, 
- se tentarmos inserir um emprestimo entre os usuarios 1 e 2 devemos ter sucesso, 
- mas se tentarmos inserir um emprestimo entre os usuarios 1 e 1000 devemos ter uma falha. 

Se ambos os comandos de inserção devem ser executados de modo 'tudo ou nada' (ou executam ambos, ou nenhum), parece que temos um caso ideal para aplicar transações aqui. A idéia é que se tudo acontecer sem problemas devemos executar um `COMMIT`, mas se algo der errado devemos excutar um `ROLLBACK`. Isso parece uma missão para `try`/`catch`!

In [39]:
try:
    db('START TRANSACTION')
    db('INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 2)')
    db('INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 1000)')
    db('COMMIT')
except Exception as e:
    print(e)
    db('ROLLBACK')

Executando query:
Executando query:
Executando query:
1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`emprestimos`.`emprestimo`, CONSTRAINT `fk_devedor` FOREIGN KEY (`id_devedor`) REFERENCES `usuario` (`id_usuario`))
Executando query:


In [40]:
db('SELECT * FROM emprestimo')

Executando query:


### Atividade

- Execute os depósitos abaixo:

| usuario        | valor |
|----------------|-------|
| Ana Soares     | 10000 |
| Antonio Reis   | 5000  |
| Paulo Oliveira | 20000 |

Dica: crie uma função `pega_id_usuario(connection, nome, sobrenome)` para achar o id de usuario, e uma função `deposito(connection, id_usuario, valor)` para facilitar a realização dos depósitos. Um depósito requer uma inserção na tabela de movimentações, e um update no saldo.

In [71]:
def pega_id_usuario_(connection, nome, sobrenome):
    comando = (
    '''SELECT 
        id_usuario 
    FROM 
        usuario 
    WHERE 
        nome = '{0}' AND sobrenome = '{1}' 
    ''' ).format(nome, sobrenome)
    with connection.cursor() as cursor:
        cursor.execute(comando)
        results = cursor.fetchall()
    if results is None:
        raise KeyError(f'Usuário {nome} {sobrenome} não encontrado')
    return results


pega_id_usuario_(connection, 'Ana', 'Soares')

[(3,)]

In [73]:
def pega_id_usuario(connection, nome, sobrenome):
    with connection.cursor() as cursor:
        cursor.execute(
            '''
            SELECT id_usuario 
            FROM usuario 
            WHERE nome = %s AND sobrenome = %s''', (nome, sobrenome))

        results = cursor.fetchall()
        
    if results is None:
        raise KeyError(f'Usuario {nome} {sobrenome} não encontrado.')
    
    return results[0][0]


In [74]:
pega_id_usuario(connection, 'Ana', 'Soares')

3

In [75]:
def deposito(connection, id_usuario, valor):
    with connection.cursor() as cursor:
        # Atualizei o saldo da pessoa depois do depósito
        query1 = '''
        UPDATE 
            usuario
        SET
            saldo = saldo + %s 
        WHERE
            id_usuario = %s
        '''
        params1 = (valor, id_usuario)
        cursor.execute(query1, params1)
        
        # Log da movimentação de conta corrente.
        
        query2 = '''
        INSERT INTO movimentacao (id_usuario, valor) VALUES (%s, %s)
        '''
        params2 = (id_usuario, valor)
        
        cursor.execute(query2, params2)
        

In [76]:
def saque(connection, id_usuario, valor):
    deposito(connection, id_usuario, -valor)
    
def start_transaction(connection):
    with connection.cursor() as cursor:
        cursor.execute('START TRANSACTION')
        
def commit(connection):
    with connection.cursor() as cursor:
        cursor.execute('COMMIT')
        
def rollback(connection):
    with connection.cursor() as cursor:
        cursor.execute('ROLLBACK')

In [78]:
clientes_valores = [
    ('Ana', 'Soares', 10000),
    ('Antonio', 'Reis', 5000),
    ('Paulo', 'Oliveira', 20000),
]

for nome, sobrenome, valor in clientes_valores:
    #print(nome, sobrenome, valor)
    try:
        start_transaction(connection)
        id_usuario = pega_id_usuario(connection, nome, sobrenome)
        deposito(connection, id_usuario, valor)
        commit(connection)
    except Exception as e:
        print(e)
        rollback(connection)

- Crie funções para consulta de saldo de conta e de valor atual de emprestimo

In [83]:
def consulta_saldo(connection, id_usuario):
    with connection.cursor() as cursor:
        query = '''
        SELECT
            saldo
        FROM
            usuario
        WHERE
            id_usuario = %s
        '''
        params = (id_usuario,)
        cursor.execute(query, params)
        
        results = cursor.fetchall()
        
    if results is None:
        raise KeyError(f'Usuário com id {id_usuario} não encontrado')
        
    return results

In [86]:
try:
    with connection.cursor() as cursor:
        nome = 'Ana'
        sobrenome = 'Soares'
        id_usuario = pega_id_usuario(connection, nome, sobrenome)
        print(f"Esse é o id do usuario {nome} {sobrenome}, {id_usuario}")
        resultado = consulta_saldo(connection, id_usuario)
        print(resultado)
except Exception as e:
    print(e)

Esse é o id do usuario Ana Soares, 3
[(Decimal('10000.00'),)]


In [87]:
def consulta_emprestimo(connection, id_credor, id_devedor):
    with connection.cursor() as cursor:
        query = '''
        SELECT
            id_emprestimo, valor
        FROM
            emprestimo
        WHERE
            id_credor = %s AND id_devedor = %s        
        '''
        params = (id_credor, id_devedor)
        cursor.execute(query, params)
        results = cursor.fetchall()
    
    if results is None:
        raise KeyError(f'Movimentação não encontrada!')
        
    return results

- Crie os seguintes contratos de emprestimo:


| Credor | Devedor | valor |
|--|--|--|
| Ana Soares | Juca Silva | 1000 |
| Ana Soares | Antonio Reis | 2000 |
| Paulo Oliveira | Juca Silva | 3000 |

Não se esqueça que um contrato de emprestimo requer:
- criação do emprestimo
- operação de emprestimo inicial
    - operação de saque da conta do credor
    - operacao de deposito na conta do devedor
    
Use o comando `SELECT LAST_INSERT_ID()` para pegar o id do último item inserido.

Dica:
- Crie uma função para criar um empréstimo de valor zero, que retorne o id do emprestimo.
- Crie uma função para realizar um saque. Se não for possível fazer o saque, lance uma exceção.
- Crie uma função para efetuar uma operação em um empréstimo dado - isso inclui fazer o saque e o depósito também.

In [101]:
def cria_emprestimo_(connection, id_credor, id_devedor):
    with connection.cursor() as cursor:
        query = '''
        INSERT INTO emprestimo (id_credor, id_devedor) VALUES (%s, %s) 
        '''
        
        params = (id_credor, id_devedor)
        
        cursor.execute(query, params)
        
        # retorna último id inserido
        cursor.execute('SELECT LAST_INSERT_ID()')
        
        result = cursor.fetchall()
        
        return result[0][0]

In [102]:
def cria_emprestimo(connection, id_credor, id_devedor):
    with connection.cursor() as cursor:
        query = '''
        INSERT INTO emprestimo (id_credor, id_devedor) VALUES (%s, %s)
        '''
        params = (id_credor, id_devedor)
        cursor.execute(query, params)
        cursor.execute('SELECT LAST_INSERT_ID()')
        result = cursor.fetchone()
        return result[0]

In [103]:
def atualiza_emprestimo(connection, id_emprestimo, valor):
    with connection.cursor() as cursor:
        # Atualiza saldo do emprestimo.
        query = '''
        UPDATE 
            emprestimo
        SET 
            valor_atual = valor_atual + %s
        WHERE 
            id_emprestimo = %s
        '''
        params = (valor, id_emprestimo)
        cursor.execute(query, params)

        # Log na tabela de operações.
        query = '''
        INSERT INTO operacao (id_emprestimo, valor) VALUES (%s, %s)
        '''
        params = (id_emprestimo, valor)
        cursor.execute(query, params)

In [108]:
dados = [
    ("Ana", "Soares", "Juca", "Silva", 1000),
    ("Ana", "Soares", "Antonio", "Reis", 2000),
    ("Paulo", "Oliveira", "Juca", "Silva", 3000),
]

for nome_credor, sobrenome_credor, \
    nome_devedor, sobrenome_devedor, \
    valor in dados:
    try:
        start_transaction(connection)
        id_credor = pega_id_usuario(
            connection,
            nome_credor,
            sobrenome_credor,
        )
        id_devedor = pega_id_usuario(
            connection,
            nome_devedor,
            sobrenome_devedor,
        )
        id_emprestimo = cria_emprestimo_(connection, id_credor, id_devedor)
        saque(connection, id_credor, valor)
        deposito(connection, id_devedor, valor)
        atualiza_emprestimo(connection, id_emprestimo, valor)
        commit(connection)
    except Exception as e:
        print(e)
        rollback(connection)

Por hoje é só!

In [109]:
connection.close()